In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import yaml
import numpy as np

from models import *
from experiment import VAEXperiment
from torchsummary import summary
import pytorch_lightning as pl

import torch.backends.cudnn as cudnn

In [3]:
configs_path = "configs/betatc_vae_sate.yaml"

with open(configs_path, 'r') as f:
    configs = yaml.safe_load(f)
    
model = vae_models[configs['model_params']['name']](**configs['model_params'])

In [4]:
my_model = model.cpu()

In [5]:
summary(my_model, input_size=(1, 64, 64), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             544
         LeakyReLU-2           [-1, 32, 32, 32]               0
            Conv2d-3           [-1, 32, 16, 16]          16,416
         LeakyReLU-4           [-1, 32, 16, 16]               0
            Conv2d-5             [-1, 32, 8, 8]          16,416
         LeakyReLU-6             [-1, 32, 8, 8]               0
            Conv2d-7             [-1, 32, 4, 4]          16,416
         LeakyReLU-8             [-1, 32, 4, 4]               0
            Linear-9                  [-1, 256]         131,328
           Linear-10                   [-1, 10]           2,570
           Linear-11                   [-1, 10]           2,570
           Linear-12                  [-1, 512]           5,632
  ConvTranspose2d-13             [-1, 32, 8, 8]           9,248
        LeakyReLU-14             [-1, 3

In [6]:
checkpoint_path = 'logs/_ckpt_epoch_83.ckpt'

checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))

In [7]:
state_dict = checkpoint['state_dict']

In [8]:
state_dict = {k.partition('model.')[2] : state_dict[k] for k in state_dict}

In [9]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [13]:
experiment = VAEXperiment(model,
                          configs['exp_params'])

In [14]:
dataloader = experiment.train_dataloader()